# 파이토치로 로지스틱 회귀 구현하기

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [4]:
W = torch.zeros((2, 1), requires_grad=True) # 크기는 2 x 1
b = torch.zeros(1, requires_grad=True)

hypothesis = 1 / (1+torch.exp(-(x_train.matmul(W)+b)))

In [5]:
print(hypothesis) # 예측값인 H(x) 출력

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)


In [6]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b)


print(hypothesis)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)


$cost(W) = -\frac{1}{n} \sum_{i=1}^{n} [y^{(i)}logH(x^{(i)}) + (1-y^{(i)})log(1-H(x^{(i)}))]$

In [7]:
-(y_train[0] * torch.log(hypothesis[0]) +
  (1 - y_train[0]) * torch.log(1 - hypothesis[0]))


tensor([0.6931], grad_fn=<NegBackward0>)

In [8]:
losses = -(y_train * torch.log(hypothesis) +
           (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


In [9]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


지금까지 비용 함수의 값을 직접 구현하였는데, 사실 파이토치에서는 로지스틱 회귀의 비용 함수를 이미 구현해서 제공하고 있습니다.

사용 방법은 torch.nn.functional as F와 같이 임포트 한 후에 F.binary_cross_entropy(예측값, 실제값)과 같이 사용하면 됩니다.

In [10]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

전체 출력 과정

In [11]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)


W = torch.zeros((2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W,b], lr=1)
num_epochs = 1000

for epoch in range(num_epochs + 1):
  hypothesis = torch.sigmoid(x_train.matmul(W) + b)
  cost = F.binary_cross_entropy(hypothesis, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, num_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [12]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis)


tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)


In [13]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction)


tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])


In [14]:
print(W)
print(b)


tensor([[3.2530],
        [1.5179]], requires_grad=True)
tensor([-14.4819], requires_grad=True)


# nn.Module로 구현하는 로지스틱 회귀

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


torch.manual_seed(1)

In [16]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [17]:
model = nn.Sequential(
    nn.Linear(2, 1),
    nn.Sigmoid()
)


model(x_train)

tensor([[0.4020],
        [0.4147],
        [0.6556],
        [0.5948],
        [0.6788],
        [0.8061]], grad_fn=<SigmoidBackward0>)

In [19]:
optimizer = optim.SGD(model.parameters(), lr=1)
num_epochs = 100

for epoch in range(num_epochs + 1):
  hypothesis = model(x_train)
  loss_func = F.binary_cross_entropy(hypothesis, y_train)

  optimizer.zero_grad()
  loss_func.backward()
  optimizer.step()

  if epoch % 10 == 0:
    pred = hypothesis >= torch.FloatTensor([0.5])
    correct_pred = pred.float() == y_train
    accuracy = correct_pred.sum().item() / len(correct_pred)

    print('Epoch {:4d}/{} Loss: {:.6f} Accuracy:{:2.2f}'.format(
        epoch, num_epochs, loss_func.item(), accuracy*100))

Epoch    0/100 Loss: 0.019825 Accuracy:100.00
Epoch   10/100 Loss: 0.019642 Accuracy:100.00
Epoch   20/100 Loss: 0.019463 Accuracy:100.00
Epoch   30/100 Loss: 0.019288 Accuracy:100.00
Epoch   40/100 Loss: 0.019115 Accuracy:100.00
Epoch   50/100 Loss: 0.018946 Accuracy:100.00
Epoch   60/100 Loss: 0.018779 Accuracy:100.00
Epoch   70/100 Loss: 0.018615 Accuracy:100.00
Epoch   80/100 Loss: 0.018455 Accuracy:100.00
Epoch   90/100 Loss: 0.018297 Accuracy:100.00
Epoch  100/100 Loss: 0.018141 Accuracy:100.00


In [20]:
print(list(model.parameters()))

[Parameter containing:
tensor([[3.3406, 1.5649]], requires_grad=True), Parameter containing:
tensor([-14.8861], requires_grad=True)]


# 클래스로 파이토치 모델 구현하기

In [21]:
class BinaryClassifier(nn.Module): #이진분류
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(2, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    return self.sigmoid(self.linear(x))


torch.manual_seed(1)

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [26]:
model = BinaryClassifier() #epoch : 100대에서 정확도 100%


optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/100 Cost: 0.576740 Accuracy 83.33333333333334%
Epoch   10/100 Cost: 0.818109 Accuracy 66.66666666666666%
Epoch   20/100 Cost: 0.582192 Accuracy 83.33333333333334%
Epoch   30/100 Cost: 0.488437 Accuracy 83.33333333333334%
Epoch   40/100 Cost: 0.402663 Accuracy 83.33333333333334%
Epoch   50/100 Cost: 0.321072 Accuracy 83.33333333333334%
Epoch   60/100 Cost: 0.247059 Accuracy 83.33333333333334%
Epoch   70/100 Cost: 0.190018 Accuracy 100.0%
Epoch   80/100 Cost: 0.158865 Accuracy 100.0%
Epoch   90/100 Cost: 0.144834 Accuracy 100.0%
Epoch  100/100 Cost: 0.134897 Accuracy 100.0%
